In [1]:
!pip install wget | tail -n 1
!pip install scikit-learn | tail -n 1
!pip install "ibm-watson-machine-learning>=1.0.310" | tail -n 1

In [ ]:
url = "https://us-south.ml.cloud.ibm.com"
apikey='enter your api key '


In [3]:
credentials = {
    "url": url,
    "apikey": apikey
}


In [4]:
import os

try:
    project_id = os.environ["PROJECT_ID"]
except KeyError:
    project_id = input("Please enter your project_id (hit enter): ")


project_id


'3b65b630-96b0-413b-bbd3-5511f839b826'

In [8]:
import types
import pandas as pd
from botocore.client import Config
import ibm_boto3
from io import StringIO # Import StringIO

cos_client = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='AcSmvalVuyr5r5PkpmIuQGX7fxkfDPw_MmSvusXFd27a',  # Your secret API key
    ibm_auth_endpoint='https://iam.cloud.ibm.com/oidc/token',
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.us-south.cloud-object-storage.appdomain.cloud'
)

bucket = 'bucket-n38x7k26jbac15c'
object_key = 'legal_sentiment_dataset (1).csv'

# Get the object content
body = cos_client.get_object(Bucket=bucket, Key=object_key)['Body']

# Read the content into a string
csv_string = body.read().decode('utf-8')

# Use StringIO to treat the string as a file
data = pd.read_csv(StringIO(csv_string))

data.head(10)

,ID,Phrase,Sentiment
0,1,The plaintiff's claims are dismissed.,-1
1,2,The contract is deemed valid and enforceable.,1
2,3,The appeal is denied due to lack of merit.,-1
3,4,The legal team submitted additional evidence.,0
4,5,The appeal is under consideration.,0
5,6,The settlement agreement is fair and reasonable.,1
6,7,The appeal is under consideration.,0
7,8,The settlement agreement is fair and reasonable.,1
8,9,The court reviewed the documentation.,0
9,10,The contract is deemed valid and enforceable.,1


In [37]:
label_map={
    -1:'negative',
     0:'netural',
     1: 'positive'
}

In [9]:
data.value_counts(['Sentiment'])

Sentiment
 1           184
-1           167
 0           149
Name: count, dtype: int64

In [10]:
from sklearn.model_selection import train_test_split
import pandas as pd

data_train, data_test, y_train, y_test = train_test_split(
    data['Phrase'],
    data['Sentiment'],
    test_size=0.3,
    random_state=33,
    stratify=data['Sentiment']
)

# Convert to DataFrames
data_train = pd.DataFrame({'Phrase': data_train, 'Sentiment': y_train})
data_test = pd.DataFrame({'Phrase': data_test, 'Sentiment': y_test})

In [11]:
from ibm_watson_machine_learning.foundation_models.utils.enums import ModelTypes

print([model.name for model in ModelTypes])

['FLAN_T5_XXL', 'FLAN_UL2', 'MT0_XXL', 'GPT_NEOX', 'MPT_7B_INSTRUCT2', 'STARCODER', 'LLAMA_2_70B_CHAT', 'LLAMA_2_13B_CHAT', 'GRANITE_13B_INSTRUCT', 'GRANITE_13B_CHAT', 'FLAN_T5_XL', 'GRANITE_13B_CHAT_V2', 'GRANITE_13B_INSTRUCT_V2', 'ELYZA_JAPANESE_LLAMA_2_7B_INSTRUCT', 'MIXTRAL_8X7B_INSTRUCT_V01_Q', 'CODELLAMA_34B_INSTRUCT_HF', 'GRANITE_20B_MULTILINGUAL']


In [12]:
model_id = ModelTypes.FLAN_T5_XXL

In [13]:
from ibm_watson_machine_learning.metanames import GenTextParamsMetaNames as GenParams

parameters = {
    GenParams.DECODING_METHOD: "greedy",
    GenParams.RANDOM_SEED: 33,
    GenParams.REPETITION_PENALTY: 1,
    GenParams.MIN_NEW_TOKENS: 1,
    GenParams.MAX_NEW_TOKENS: 1
}

In [ ]:
from ibm_watson_machine_learning.foundation_models import Model

# Ensure these are correctly defined
model_id = "google/flan-ul2" # Example model ID
parameters = {
    "decoding_method": "greedy",
    "min_new_tokens": 10,
    "max_new_tokens": 50
}
credentials = {
    "apikey": "enter your apikey", # Replace with your actual API key
    "url": "https://us-south.ml.cloud.ibm.com" # Replace with your WML service endpoint URL
}
project_id = "3b65b630-96b0-413b-bbd3-5511f839b826" # Replace with your actual project ID

try:
    model = Model(
        model_id=model_id,
        params=parameters,
        credentials=credentials,
        project_id=project_id
    )
    print("Model object initialized successfully!")
    # You can now use the 'model' object for inference
except Exception as e:
    print(f"An error occurred during model initialization: {e}")

Model object initialized successfully!


/opt/conda/envs/Python-RT24.1/lib/python3.11/site-packages/ibm_watson_machine_learning/foundation_models/utils/utils.py:273: LifecycleWarning: Model 'google/flan-ul2' is in deprecated state from 2025-05-28 until 2025-07-30. IDs of alternative models: None. Further details: https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/fm-model-lifecycle.html?context=wx&audience=wdp
  warnings.warn(default_warning_template.format(


In [17]:
model.get_details()

{'model_id': 'google/flan-ul2',
 'label': 'flan-ul2-20b',
 'provider': 'Google',
 'source': 'Hugging Face',
 'functions': [{'id': 'autoai_rag'}, {'id': 'text_generation'}],
 'short_description': 'flan-ul2 is an encoder decoder model based on the T5 architecture and instruction-tuned using the Fine-tuned Language Net.',
 'long_description': 'flan-ul2 (20B) is an encoder decoder model based on the T5 architecture and instruction-tuned using the Fine-tuned Language Net (FLAN). Compared to the original UL2 model, flan-ul2 (20B) is more usable for few-shot in-context learning because it was trained with a three times larger receptive field. flan-ul2 (20B) outperforms flan-t5 (11B) by an overall relative improvement of +3.2%.',
 'terms_url': 'https://huggingface.co/google/flan-ul2/blob/main/README.md',
 'input_tier': 'class_3',
 'output_tier': 'class_3',
 'number_params': '20b',
 'min_shot_size': 0,
 'task_ids': ['question_answering',
  'summarization',
  'retrieval_augmented_generation',
  

In [18]:
instruction = """Determine the sentiment of the sentence.
Use either 'positive', 'negative', 'neutral'. Use the provided examples as a template."""

In [19]:
zero_shot_inputs = [{"input": text} for text in data_test['Phrase']]
for i in range(2):
    print(f"The sentence example {i+1} is:\n\t {zero_shot_inputs[i]['input']}\n")

The sentence example 1 is:
	 The legal team submitted additional evidence.

The sentence example 2 is:
	 The appeal is denied due to lack of merit.



In [20]:
data_train_and_labels = data_train.copy()
data_train_and_labels['Sentiment'] = y_train

In [21]:
few_shot_example = []
few_shot_examples = []

In [22]:
for phrase, sentiment in data_train_and_labels \
    .groupby('Sentiment') \
    .apply(lambda x: x.sample(2)).values:
    
    few_shot_example.append(f"\tsentence:\t{phrase}\n\tsentiment: {sentiment}\n")

few_shot_examples = '\n'.join(few_shot_example)

In [23]:
few_shot_inputs_ = [{"input": text} for text in data_test['Phrase'].values]

In [25]:
for i in range(2):
    print(f"The sentence example {i+1} is:\n {few_shot_inputs_[i]['input']}\n")
    print(f"\tSentiment: {y_test.iloc[i]}\n")


The sentence example 1 is:
 The legal team submitted additional evidence.

	Sentiment: 0

The sentence example 2 is:
 The appeal is denied due to lack of merit.

	Sentiment: -1



In [26]:
results = []
for inp in few_shot_inputs_[:2]:
    results.append(
        model.generate(" ".join([instruction + few_shot_examples[0], inp['input']]))["results"][0]
    )


In [27]:
import json
print(json.dumps(results, indent=2))

[
  {
    "generated_text": "neutral.. . . . . . . . . . . . . . . . . . . . . . . . ",
    "generated_token_count": 50,
    "input_token_count": 42,
    "stop_reason": "max_tokens"
  },
  {
    "generated_text": "negative.. . . . . . . . . . . . . . . . . . . . . . . . ",
    "generated_token_count": 50,
    "input_token_count": 45,
    "stop_reason": "max_tokens"
  }
]


In [35]:
y_pred = [result['generated_text'] for result in results]

In [38]:
y_true = [label_map[label] for label in y_test.values[:2]]

In [ ]:
few_shot_examples_str = '\n'.join(few_shot_example) 

results = []
for inp_dict in few_shot_inputs_: # Iterate over ALL test inputs
    # Construct the full prompt for each test input
    full_prompt = f"{instruction}\n\n{few_shot_examples_str}\n\nText: {inp_dict['input']}\nSentiment:"

    try:
        # Generate prediction
        generated_text = model.generate_text(prompt=full_prompt)
        results.append({'generated_text': generated_text.strip()}) 
    except Exception as e:
        print(f"Error generating for input: {inp_dict['input']} - {e}")
        results.append({'generated_text': 'ERROR'}) 

y_pred = [result['generated_text'] for result in results]

In [48]:
cleaned_y_pred = []
for pred in y_pred:
    pred_lower = pred.lower().strip()
    if 'positive' in pred_lower:
        cleaned_y_pred.append('positive')
    elif 'negative' in pred_lower:
        cleaned_y_pred.append('negative')
    elif 'neutral' in pred_lower or 'netural' in pred_lower: # Handle typo if model generates it
        cleaned_y_pred.append('neutral')
    else:
        cleaned_y_pred.append('unknown') # Handle cases where model doesn't give expected output

In [41]:
y_true_strings = [label_map[label] for label in y_test.values] # Apply to full y_test

In [47]:
data_train, data_test, y_train, y_test = train_test_split(
    data['Phrase'],
    data['Sentiment'],
    test_size=0.3,
    random_state=33,
    stratify=data['Sentiment']
)

# Convert to DataFrames
data_train = pd.DataFrame({'Phrase': data_train, 'Sentiment': y_train})
data_test = pd.DataFrame({'Phrase': data_test, 'Sentiment': y_test})


y_true_strings = [label_map[label] for label in data_test['Sentiment'].values] # <--- FIX IS HERE

# --- Verify lengths again (after the fix) ---
print(f"\nLength of data_test['Phrase']: {len(data_test['Phrase'])}")
print(f"Length of data_test['Sentiment']: {len(data_test['Sentiment'])}") # Should now be 150
print(f"Length of y_true_strings: {len(y_true_strings)}") # Should now be 150
print(f"Length of cleaned_y_pred: {len(cleaned_y_pred)}") # Should be 150 if inference worked fully

# --- Create DataFrame and Save ---
# Now, all lengths should match, and this will execute without error
results_df = pd.DataFrame({
    'Phrase': data_test['Phrase'].tolist(),
    'True_Sentiment': y_true_strings,
    'Predicted_Sentiment': cleaned_y_pred
})

csv_file_name = 'legal_sentiment_analysis_results.csv'
results_df.to_csv(csv_file_name, index=False)
print(f"\nFull results saved to {csv_file_name}")


from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

print("\n--- Evaluation Results ---")
print("Accuracy:", accuracy_score(y_true_strings, cleaned_y_pred))
print("\nClassification Report:\n", classification_report(y_true_strings, cleaned_y_pred))
print("\nConfusion Matrix:\n", confusion_matrix(y_true_strings, cleaned_y_pred))


Length of data_test['Phrase']: 150
Length of data_test['Sentiment']: 150
Length of y_true_strings: 150
Length of cleaned_y_pred: 150

Full results saved to legal_sentiment_analysis_results.csv

--- Evaluation Results ---
Accuracy: 0.38666666666666666

Classification Report:
               precision    recall  f1-score   support

    negative       1.00      0.58      0.73        50
     netural       0.00      0.00      0.00        45
     neutral       0.00      0.00      0.00         0
    positive       1.00      0.53      0.69        55
     unknown       0.00      0.00      0.00         0

    accuracy                           0.39       150
   macro avg       0.40      0.22      0.28       150
weighted avg       0.70      0.39      0.50       150


Confusion Matrix:
 [[29  0 12  0  9]
 [ 0  0 45  0  0]
 [ 0  0  0  0  0]
 [ 0  0 26 29  0]
 [ 0  0  0  0  0]]


/opt/conda/envs/Python-RT24.1/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/envs/Python-RT24.1/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/envs/Python-RT24.1/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/envs

In [46]:
import pandas as pd
# Create a DataFrame from your results
results_df = pd.DataFrame({
    'Phrase': data_test['Phrase'].tolist(), # Use the full test phrases
    'True_Sentiment': [lable_map[label] for label in y_test.values], # Use the full y_test
    'Predicted_Sentiment': cleaned_y_pred # This should contain predictions for the full test set
})

# Save to CSV
results_df.to_csv('sentiment_analysis_results.csv', index=False)
print("Results saved to sentiment_analysis_results.csv")

Results saved to sentiment_analysis_results.csv
